In [74]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')

In [75]:
data = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
data.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
data.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
data

,Member,Address,City,Postal Code
0,Alberta Conference SDA Church,5816 Hwy 2A,Lacombe,T4L 2G5
1,Alberta Health Services,48 Quarry Park Blvd. SE,Calgary,T2C 5P2
2,Alberta Innovates,"1500, 10104 - 103 Avenue",Edmonton,T5J 0H8
3,Alberta University of the Arts,1407 - 14th Avenue NW,Calgary,T2N 4R3
4,Aspen View School Division,3600 - 48 Avenue,Athabasca,T9S 1M8
...,...,...,...,...
94,University of Lethbridge,4401 University Drive,Lethbridge,T1K 3M4
95,Wetaskiwin Regional Public Schools,5704 - 51 Street,Wetaskiwin,T9A 3G1
96,Whipcord Ltd,3528 30 Street North,Lethbridge,T1H 6Z4
97,Wild Rose School Division No.66,4912 - 43 Street,Rocky Mountain House,T4T 1P4


In [76]:
lats = []
lons = []
for i in data['Postal Code']:
    lats.append(nomi.query_postal_code(i)['latitude'])
    lons.append(nomi.query_postal_code(i)['longitude'])
data['lat'] = lats
data['lon'] = lons
data['geometry'] = geopandas.points_from_xy(data['lat'], data['lon'])
data

,Member,Address,City,Postal Code,lat,lon,geometry
0,Alberta Conference SDA Church,5816 Hwy 2A,Lacombe,T4L 2G5,52.4668,-113.7353,POINT (52.467 -113.735)
1,Alberta Health Services,48 Quarry Park Blvd. SE,Calgary,T2C 5P2,50.9870,-113.9634,POINT (50.987 -113.963)
2,Alberta Innovates,"1500, 10104 - 103 Avenue",Edmonton,T5J 0H8,53.5428,-113.4974,POINT (53.543 -113.497)
3,Alberta University of the Arts,1407 - 14th Avenue NW,Calgary,T2N 4R3,51.0621,-114.1159,POINT (51.062 -114.116)
4,Aspen View School Division,3600 - 48 Avenue,Athabasca,T9S 1M8,54.7169,-113.2854,POINT (54.717 -113.285)
...,...,...,...,...,...,...,...
94,University of Lethbridge,4401 University Drive,Lethbridge,T1K 3M4,49.6511,-112.8351,POINT (49.651 -112.835)
95,Wetaskiwin Regional Public Schools,5704 - 51 Street,Wetaskiwin,T9A 3G1,52.9668,-113.3687,POINT (52.967 -113.369)
96,Whipcord Ltd,3528 30 Street North,Lethbridge,T1H 6Z4,49.7000,-112.8186,POINT (49.700 -112.819)
97,Wild Rose School Division No.66,4912 - 43 Street,Rocky Mountain House,T4T 1P4,52.3668,-114.9188,POINT (52.367 -114.919)


In [79]:
fig = go.Figure(data=go.Scattergeo(
    lat = data['lat'],
    lon = data['lon'],
    text = data['Member']
)
               )

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
# fig.show()
fig.write_html('membermap.html')